In [1]:
from selenium import webdriver
import time
import argparse
import os
import re
import numpy as np
import pandas as pd
import bs4
import requests
import random
from selenium.common.exceptions import NoSuchElementException
import datetime

In [3]:
companies = pd.read_csv('/Users/ZiyuChen/assignment/linkedin-web-scraper/companies.csv')

In [11]:
companies.iloc[0]['officer_linkedin']

'https://www.linkedin.com/in/matthew-franklin-product/'

In [15]:
def set_driver():
    # Define Chrome options to open the window in maximized mode
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")

    #Anti-web-scraping
    options.add_experimental_option('excludeSwitches', ['enable-automation'])

    # Initialize the Chrome webdriver and open the URL
    driver = webdriver.Chrome(options=options, executable_path='/Users/ZiyuChen/Desktop/chromedriver')
    
    return driver

In [16]:
def load_page(url, pause_time = 1):
    driver.get(url)

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    # Record the starting time
    start = datetime.datetime.now()

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # wait to load page
        time.sleep(pause_time)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height: # which means end of page
            break

        # update the last height
        last_height = new_height
 
    driver.execute_script("window.scrollTo(0, 0);")
    
    # Record the end time, then calculate and print the total time
    end = datetime.datetime.now()
    delta = end-start
    print("Took {} to fully load {}".format(delta, url))

In [37]:
def login(email, password):
    # Load Linked-in's signin page:
    load_page('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')
    # Select the username input
    username_input = driver.find_element_by_id('username')
    # Fill in the user email
    username_input.clear()
    username_input.send_keys(email)
    # Select the password input
    password_input = driver.find_element_by_id('password')
    # Fill in the password
    password_input.clear()
    password_input.send_keys(password)
    # Select the signin button:
    signin_button = driver.find_element_by_class_name('login__form_action_container ').find_element_by_class_name('btn__primary--large')
    signin_button.click()

def get_officer_data(company):
    load_page(company['officer_linkedin'])
    get_officer_experience_list()
    load_page(company['officer_linkedin']+'detail/photo/')
    get_officer_photo()
    

def get_officer_photo():
    try:
        profile_url = driver.find_element_by_class_name('pv-member-photo-modal__profile-image').get_attribute("src")
    except NoSuchElementException:
        profile_url = None
    print(profile_url)

def get_officer_experience_list():
    try:
        experience_list_el = driver.find_element_by_id('experience-section').find_elements_by_class_name('pv-profile-section__list-item')
        for experience_el in experience_list_el:
            experience = get_officer_experience(experience_el)
            print(experience)
    except NoSuchElementException:
        print('Cannot access officer experience')
        pass

def get_officer_experience(experience_el):
    experience = {
        'job_company': None,
        'job_title': None,
        'job_time': None,
        'job_duration': None,
        'job_location': None
    }
    experience_el = experience_el.find_element_by_class_name('pv-entity__summary-info--background-section')
    
    # Get company
    try:
        company = experience_el.find_element_by_class_name('pv-entity__secondary-title').text
        experience['job_company'] = company
    except NoSuchElementException:
        pass
    
    # Get title
    try:
        title = experience_el.find_element_by_class_name('t-16').text
        experience['job_title'] = title
    except NoSuchElementException:
        pass
    
    # Get time
    try:
        time = experience_el.find_element_by_class_name('pv-entity__date-range').text.replace('Dates Employed', '').strip()
        experience['job_time'] = time
    except NoSuchElementException:
        pass
    
    # Get duration
    try:
        duration = experience_el.find_element_by_class_name('pv-entity__bullet-item-v2').text
        experience['job_duration'] = duration
    except NoSuchElementException:
        pass
    
    # Get location
    try:
        location = experience_el.find_element_by_class_name('pv-entity__location').text.replace('Location', '').strip()
        experience['job_location'] = location
    except NoSuchElementException:
        pass
    
    return experience
        

In [38]:
driver = set_driver()
login()
get_officer_data(companies.iloc[9])

/Users/ZiyuChen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


Took 0:00:01.029676 to fully load https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin
Took 0:00:03.064331 to fully load https://www.linkedin.com/in/daniil-voloshin-10467277/
{'job_company': 'EMIBA', 'job_title': 'Chief Executive Officer', 'job_time': 'Sep 2017 – Present', 'job_duration': '2 yrs 6 mos', 'job_location': 'Chelyabinsk, Russia'}
{'job_company': 'Castle Family Office', 'job_title': 'Partner', 'job_time': 'May 2017 – Aug 2019', 'job_duration': '2 yrs 4 mos', 'job_location': 'Central Singapore'}
Took 0:00:01.023568 to fully load https://www.linkedin.com/in/daniil-voloshin-10467277/detail/photo/
https://media-exp1.licdn.com/dms/image/C4E03AQHMLTGWUrnZuQ/profile-displayphoto-shrink_800_800/0?e=1588204800&v=beta&t=sYDPuopm_j05cNEtQkHl1vnHteMQ3VqkHYL_kKTrsEQ
